# 📘 Introduction to Transformer and Attention Mechanism

This notebook will walk you through:
- What is Attention?
- Self-Attention Mechanism
- The Transformer Architecture
- Multi-Head Attention
- Position-wise Feed-Forward Networks
- Positional Encoding
- Code Examples

## 🔍 1. What is Attention?

In the context of deep learning, **attention** is a technique that allows models to dynamically focus on relevant parts of the input sequence.

Consider translating:
> "The animal didn't cross the street because it was too tired."

When deciding what "it" refers to, the model must **attend** to the word "animal".

### The Core Idea:
Instead of encoding the entire sentence into a fixed vector (as in traditional RNN/Seq2Seq), attention allows the model to **weigh** the importance of each word in the sequence.

## ✨ 2. Scaled Dot-Product Attention (Enhanced Explanation)

### Core Concept Visualization

Imagine you're a teacher grading essays:
- **Query (Q)**: Your grading rubric (what you're looking for)
- **Key (K)**: Each essay's thesis statement
- **Value (V)**: The actual essay content

```mermaid
graph LR
    Q[Query] -->|Compare| K[Key]
    K -->|Generate| W[Weights]
    W -->|Weighted Sum| V[Value]
    V --> O[Output]
```

### Step-by-Step Calculation

1. **Similarity Scores**: 
   ```python
   raw_scores = Q @ K.T  # Matrix multiplication
   ```
   
2. **Scaling**:
   ```python
   scaled_scores = raw_scores / sqrt(d_k)  # d_k = dimension of keys
   ```
   
3. **Softmax Normalization**:
   ```python
   attention_weights = softmax(scaled_scores)
   ```
   
4. **Context Vector**:
   ```python
   output = attention_weights @ V
   ```

### Why Scaling Matters
Without scaling (`1/sqrt(d_k)`), when dimensions are large:
- Dot products grow extremely large
- Softmax gradients become vanishingly small
- Model can't learn effectively

### Complete Formula

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

## 🔁 3. Self-Attention (Detailed Breakdown)

### Self-Attention vs Regular Attention

| Feature          | Self-Attention         | Regular Attention      |
|------------------|-----------------------|-----------------------|
| Query Source     | Input sequence itself | External (e.g., decoder) |
| Key/Value Source | Input sequence itself | External source       |
| Directionality   | Bidirectional         | Often unidirectional  |

### Key Properties

1. **Long-Range Dependencies**:
   ```text
   Sentence: "The cat, which was hungry despite eating recently, meowed loudly."
   Relation: "cat" ↔ "meowed" (distant but directly connected)
   ```
   
2. **Parallel Computation**:
   All position pairs computed simultaneously (unlike RNNs)
   
3. **Contextual Encoding**:
   Each word's representation changes based on entire context

### Computation Process

```python
# Input embeddings (n × d_model)
X = [word1_embed, word2_embed, ...]  

# Learnable projections
Q = X @ W_Q  # (n × d_k)
K = X @ W_K  # (n × d_k)
V = X @ W_V  # (n × d_v)

# Self-attention calculation
attention = softmax((Q @ K.T)/sqrt(d_k)) @ V  # (n × d_v)
```

### Practical Example

In [ ]:
import torch
import torch.nn.functional as F

# Sample sentence embeddings (3 words, 4 dimensions each)
X = torch.tensor([
    [1.0, 0.2, -0.5, 0.3],  # Word 1 ("The")
    [0.5, 1.2, 0.1, -0.7], # Word 2 ("cat")
    [-0.3, 0.8, 1.1, 0.4]   # Word 3 ("meowed")
], dtype=torch.float32)

# Projection matrices (randomly initialized)
W_Q = torch.randn(4, 3)
W_K = torch.randn(4, 3)
W_V = torch.randn(4, 2)

# Compute Q, K, V
Q = X @ W_Q
K = X @ W_K
V = X @ W_V

# Self-attention calculation
d_k = Q.size(-1)
scores = (Q @ K.T) / torch.sqrt(torch.tensor(d_k))
weights = F.softmax(scores, dim=-1)
output = weights @ V

print("Attention Weights:\n", weights)
print("\nOutput (Contextual Embeddings):\n", output)

### Interpretation
- **Attention Weights**: Shows how much each word attends to others
- **Output**: New embeddings where each word contains information from relevant context words

> Note: Real implementations use multi-head attention and masking for decoder

## 🧠 4. Transformer Architecture (Encoder-Decoder)

The original Transformer consists of:

### Encoder Block:
- Multi-head self-attention
- Feed-forward layer
- Residual + LayerNorm

### Decoder Block:
- Masked multi-head self-attention
- Multi-head attention over encoder output
- Feed-forward layer
- Residual + LayerNorm

```mermaid
graph TD
    A[Input] --> B[Positional Encoding]
    B --> C[Encoder Blocks]
    C --> D[Context Vector]
    D --> E[Decoder Blocks]
    E --> F[Output]
```

## 🧩 5. Multi-Head Attention

Instead of one single attention function, the Transformer uses **multiple attention heads**:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(head_1, ..., head_h)W^O
$$

where each head is:
$$
head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

### Why Multiple Heads?
- Each head learns different attention patterns
- Example heads might focus on:
  - Positional relationships
  - Syntactic features
  - Semantic relationships
- Combines benefits of CNN's multiple filters

## 🔄 6. Feed-Forward Layer

Each position in the sequence is passed through the same feed-forward network:

$$
\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2
$$

### Key Properties:
- Applied independently to each position
- Usually expands dimension then reduces (e.g., 512 → 2048 → 512)
- Provides additional non-linearity
- Sometimes called "position-wise" FFN

## 🌀 7. Positional Encoding

Since the Transformer has no recurrence, it adds **positional encodings** to input embeddings:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

### Visualized Pattern:
<img src="https://jalammar.github.io/images/t/transformer_positional_encoding_example.png" width="400">

### Why This Works:
- Unique encoding for each position
- Relative positions can be linearly attended to
- Stable across sequence lengths

## 💡 8. Complete Self-Attention Implementation

In [ ]:
import math
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_model, d_k):
        super().__init__()
        self.W_Q = nn.Linear(d_model, d_k)
        self.W_K = nn.Linear(d_model, d_k)
        self.W_V = nn.Linear(d_model, d_k)
        self.d_k = d_k
        
    def forward(self, X):
        Q = self.W_Q(X)  # (batch_size, seq_len, d_k)
        K = self.W_K(X)  # (batch_size, seq_len, d_k)
        V = self.W_V(X)  # (batch_size, seq_len, d_v)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        weights = torch.softmax(scores, dim=-1)
        output = torch.matmul(weights, V)
        
        return output, weights

# Example usage
d_model = 512
d_k = 64
batch_size = 4
seq_len = 10

attention = SelfAttention(d_model, d_k)
X = torch.rand(batch_size, seq_len, d_model)
output, attn_weights = attention(X)

print(f"Input shape: {X.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

## 🏁 Summary

- **Attention Mechanisms**: Enable dynamic focus on relevant input parts
- **Self-Attention**: Captures all pairwise relationships in a sequence
- **Transformer Architecture**:
  - Encoder: Processes input with self-attention and FFN
  - Decoder: Generates output with masked self-attention
- **Key Innovations**:
  - Multi-head attention
  - Positional encodings
  - Residual connections
- **Impact**: Foundation for modern models like BERT, GPT, T5, etc.

Next Steps:
- Experiment with multi-head attention
- Implement a full Transformer
- Explore variants (Sparse Attention, Linear Attention)